In [3]:
import pandas as pd
import numpy as np

from os.path import join

from collections import Counter

import re

from datetime import datetime

In [4]:
basePath = join("..", "data", "cleaned")

In [8]:
df = pd.read_csv(join(basePath, "Zepto.csv"))

In [9]:
df["Location"].unique()

array(['Dharampeth', 'Mahal', 'Ayodhya Nagar', 'Manish Nagar',
       'Sai Mandir'], dtype=object)